#HW 4 - QMSS 5073 Machine Learning for Social Science
##Cindy Chen, cjc2279

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
! pip install -q aimodelshare

     |████████████████████████████████| 121 kB 5.5 MB/s 
     |████████████████████████████████| 146 kB 13.1 MB/s 
     |████████████████████████████████| 96 kB 4.0 MB/s 
     |████████████████████████████████| 131 kB 50.4 MB/s 
     |████████████████████████████████| 7.7 MB 40.9 MB/s 
     |████████████████████████████████| 12.7 MB 21.9 MB/s 
     |████████████████████████████████| 300 kB 50.5 MB/s 
     |████████████████████████████████| 178 kB 51.6 MB/s 
     |████████████████████████████████| 22.3 MB 60.2 MB/s 
     |████████████████████████████████| 127 kB 45.5 MB/s 
     |████████████████████████████████| 435 kB 53.4 MB/s 
     |████████████████████████████████| 271 kB 51.6 MB/s 
     |████████████████████████████████| 4.9 MB 42.2 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 87 kB 7.0 MB/s 
ERROR: pip's dependency

In [3]:
#import my common libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# fix random seed for reproducibility
np.random.seed(7)

##Part 1: Build a classification model using text data

ANSWER:
> I built three classification models and ran a LogisticRegression on the data.  I used CountVectorizer and TfidfVectorizer, and tried different n-gram and document feature parameters.  Out of my three trials, my second model using the following parameters, vectorizers, and other considerations:

* TfidfVectorizer
* the inclusion of stop words
* A C parameter of 100 (in other words, more freedom in the decision boundary)
* inclusion of single words and bi-grams

> yielded the highest f1 score of 0.96. 

> It is important to note that f-1 performance was critical to my evaluation of my model since it considers false positives and false negatives in determining accuracy.

Load aimodelshare

In [4]:
import aimodelshare as ai
X_train, X_test, y_train_labels, y_test_labels, example_data, lstm_model, lstm_model2 = ai.import_quickstart_data("clickbait")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


Step 1. Preprocess my data by using **CountVectorizer** to vectorize the clickbait text

In [5]:
vect = CountVectorizer(stop_words = "english").fit(X_train)
X_train_2 = vect.transform(X_train)

#fit X_test using transform
X_test_2 = vect.transform(X_test)

Recode my y-labels into binary 1 and 0 values. This is necessary if I want to use sklearn to evaluate my performance on an f1 score.

In [6]:
current_vals_1 = [(y_train_labels == 'clickbait'), (y_train_labels == 'not clickbait')]
current_vals_2 = [(y_test_labels == 'clickbait'), (y_test_labels == 'not clickbait')]
recoded_vals = [1, 0]

y_train_labels = np.select(current_vals_1, recoded_vals, default = np.nan)
y_test_labels = np.select(current_vals_2, recoded_vals, default = np.nan)

Review my data before I run any logistic regressions.

In [7]:
feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("Every 500th feature:\n{}".format(feature_names[::500]))
print("X data shape: ", X_train.shape, X_test.shape)
print("Y data shape: ", y_train_labels.shape, y_test_labels.shape)

Number of features: 20066
Every 500th feature:
['00', 'accident', 'anchor', 'austrian', 'belonging', 'bradley', 'canes', 'choppy', 'conducting', 'crowne', 'denomination', 'donald', 'emerges', 'fabiola', 'flush', 'genuine', 'guns', 'hipster', 'imprisons', 'jackets', 'kmart', 'limbo', 'marble', 'militants', 'nacional', 'ode', 'parks', 'plaintiffs', 'professors', 'ravaged', 'resisting', 'rut', 'separate', 'slater', 'stacey', 'superstitious', 'texting', 'tripathi', 'using', 'website', 'zealand']
X data shape:  (24979,) (6245,)
Y data shape:  (24979,) (6245,)


In [8]:
scores = cross_val_score(LogisticRegression(), X_train_2, y_train_labels, cv = 5, scoring = "f1")
print("Mean cross-validation accuracy: {:.3f}".format(np.mean(scores)))

Mean cross-validation accuracy: 0.949


Run GridSearchCV

In [9]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
gridsearch = GridSearchCV(LogisticRegression(max_iter = 10000), param_grid = param_grid, cv = 5, scoring= "f1")
gridsearch.fit(X_train_2, y_train_labels)
y_pred = gridsearch.predict(X_test_2)

print("Test score: {:.2f}".format(gridsearch.score(X_test_2, y_test_labels)))
print("Best cross-validation score: {:.2f}".format(gridsearch.best_score_))
print("Best parameters: ", gridsearch.best_params_)
print(classification_report(y_test_labels, y_pred))

Test score: 0.95
Best cross-validation score: 0.95
Best parameters:  {'C': 10}
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96      3223
         1.0       0.95      0.95      0.95      3022

    accuracy                           0.95      6245
   macro avg       0.95      0.95      0.95      6245
weighted avg       0.95      0.95      0.95      6245



Step 2. Preprocess my data by using **TfidfVectorizer** (using an ngram_range of 1 and 2) to vectorize the clickbait text

In [10]:
tfvect = TfidfVectorizer(ngram_range=(1,2), stop_words = "english").fit(X_train)
X_train_tf2 = tfvect.transform(X_train)

#fit X_test using transform
X_test_tf2 = tfvect.transform(X_test)

In [11]:
feature_names = tfvect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("Every 500th feature:\n{}".format(feature_names[::500]))
print("X data shape: ", X_train_tf2.shape, X_test_tf2.shape)
print("Y data shape: ", y_train_labels.shape, y_test_labels.shape)

Number of features: 117416
Every 500th feature:
['00', '12 disney', '15 nearly', '18 adorable', '20 life', '21 candid', '24 mind', '290', '43 marathons', '89 dead', 'access saudi', 'actress brittany', 'administration changes', 'afghan troops', 'ahs coven', 'albus dumbledore', 'america biggest', 'anil', 'ants protect', 'architect dies', 'art decor', 'astronauts let', 'australia lose', 'away ugandan', 'bake booze', 'bargaining power', 'bbc play', 'begins wobbly', 'best sweater', 'billion bailout', 'blasting mtv', 'bogosian writing', 'born terrorist', 'brazilian soccer', 'british fast', 'buffay supposed', 'busts', 'called piecaken', 'canadians won', 'cardinals', 'castroneves wins', 'celebrity ex', 'change omani', 'cheaper', 'chilean earthquake', 'christmas decorations', 'claims men', 'close ranks', 'colleagues gives', 'commander face', 'concerned situation', 'considers loyalty', 'cooker wrong', 'councilors', 'crash brazil', 'crisis short', 'cup rugby', 'daily sunday', 'dawson actually', '

In [12]:
scores = cross_val_score(LogisticRegression(), X_train_tf2, y_train_labels, cv = 5, scoring = "f1")
print("Mean cross-validation accuracy: {:.3f}".format(np.mean(scores)))

Mean cross-validation accuracy: 0.937


In [13]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
gridsearch = GridSearchCV(LogisticRegression(max_iter = 10000), param_grid = param_grid, cv = 5, scoring= "f1")
gridsearch.fit(X_train_tf2, y_train_labels)
y_pred_tf = gridsearch.predict(X_test_tf2)

print("Test score: {:.2f}".format(gridsearch.score(X_test_tf2, y_test_labels)))
print("Best cross-validation score: {:.2f}".format(gridsearch.best_score_))
print("Best parameters: ", gridsearch.best_params_)
print(classification_report(y_test_labels, y_pred_tf))

Test score: 0.96
Best cross-validation score: 0.95
Best parameters:  {'C': 100}
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      3223
         1.0       0.95      0.96      0.96      3022

    accuracy                           0.96      6245
   macro avg       0.96      0.96      0.96      6245
weighted avg       0.96      0.96      0.96      6245



Step 1. Preprocess my data by using **TfidfVectorizer** to vectorize the clickbait text

In [14]:
tf2vect = TfidfVectorizer(ngram_range=(1,3), min_df = 5, stop_words = "english").fit(X_train)
X_train_tf3 = tf2vect.transform(X_train)

#fit X_test using transform
X_test_tf3 = tf2vect.transform(X_test)

In [15]:
feature_names = tf2vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("Every 500th feature:\n{}".format(feature_names[::500]))
print("X data shape: ", X_train_tf3.shape, X_test_tf3.shape)
print("Y data shape: ", y_train_labels.shape, y_test_labels.shape)

Number of features: 6932
Every 500th feature:
['000', 'alonso', 'boycott', 'complaints', 'don want', 'flying', 'holiday', 'kosovo', 'mental', 'panda', 'raccoon', 'seek', 'sunny philadelphia', 'unexpected']
X data shape:  (24979, 6932) (6245, 6932)
Y data shape:  (24979,) (6245,)


In [16]:
scores = cross_val_score(LogisticRegression(), X_train_tf3, y_train_labels, cv = 5, scoring = "f1")
print("Mean cross-validation accuracy: {:.3f}".format(np.mean(scores)))

Mean cross-validation accuracy: 0.948


In [17]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
gridsearch = GridSearchCV(LogisticRegression(max_iter = 10000), param_grid = param_grid, cv = 5, scoring= "f1")
gridsearch.fit(X_train_tf3, y_train_labels)
y_pred_tf = gridsearch.predict(X_test_tf3)

print("Test score: {:.2f}".format(gridsearch.score(X_test_tf3, y_test_labels)))
print("Best cross-validation score: {:.2f}".format(gridsearch.best_score_))
print("Best parameters: ", gridsearch.best_params_)
print(classification_report(y_test_labels, y_pred_tf))

Test score: 0.95
Best cross-validation score: 0.95
Best parameters:  {'C': 10}
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.95      3223
         1.0       0.96      0.95      0.95      3022

    accuracy                           0.95      6245
   macro avg       0.95      0.95      0.95      6245
weighted avg       0.95      0.95      0.95      6245



##Part 2: Build a predictive neural network using Keras

####Include code for selecting the number of hidden units using GridSearchCV and evaluation on a test-set.  Describe the differences in the predictive accuracy of models with different numbers of hidden units.  Describe the predictive strength of your best model.  Be sure to explain your choice and evaluate this model using the test set.

**ANSWER:**
> In using GridSearch for 1, 3, 5, 10, and 20 hidden nodes, my best model employed 5 nodes in the first layer.  In using mean test scores (which are based on my training data) in my GridSearch to evaluate the predictive accuracy of my models with different numbers of hidden units, my model with 20 nodes had a mean predictive accuracy of 0.851.  The next three models in terms of accuracy were quite similar in scores: my model with 10 hidden nodes had a mean predictive accuracy of 0.721, followed by my mdoel with 3 hidden nodes with a mean predictive accuracy of 0.707, and then my model with 5 hidden nodes (predictive accuracy of 0.696).  Finally, my worst performing model was my model with one hidden unit, where the mean predictive accuracy was 0.465.

> In running GridSearchCV, my best model involved **20** hidden units and yielded a **training predictive accuracy of 0.851**.  This was the model that I chose as my final model. When I evaluated this same model on my test set, I encountered a test score accuracy of **0.924**.  I chose the number of epochs to be 25, because I did not want to overfit, and this strategy proved to be effective since my predictive accuracy was higher on my test set than my training data.  I also used a validation set (using 20% of my data) to help me tune my models.

> As an aside, I noticed that scaling my data dramatically improved the performance of my neural network prediction accuracy.

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, make_scorer

iris_data = pd.read_csv("http://vincentarelbundock.github.io/Rdatasets/csv/datasets/iris.csv")
iris_data.shape

(150, 6)

Divide my data into training, testing, and validation sets.  Scale my X data as well.

In [19]:
#set up my data so I can create train and test sets
iris_X = iris_data.drop("Species", axis = 1)
iris_y = iris_data['Species']

#turn my labels into dummy variables
iris_y_converted = pd.get_dummies(iris_y)
#iris_y_converted = to_categorical(iris_y, num_classes = 3)

#train test split process
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y_converted)

#scale my X data
sc = StandardScaler()
sc.fit(X_train)
X_test_scaled = sc.transform(X_test)
X_train_scaled = sc.transform(X_train)

print("X train shape: ", X_train_scaled.shape)

X train shape:  (112, 5)


Create 2 hidden layers.

In [56]:
#create two hidden layers
model = Sequential()
model.add(Dense(3, activation='relu', input_dim=5))
model.add(Dense(3, activation='softmax', input_dim=5))
optimizer = SGD(learning_rate = 0.1)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics='AUC')

model.fit(X_train_scaled, y_train, epochs=10, batch_size=128, validation_split=.2)

score = model.evaluate(X_test_scaled, y_test, batch_size=128) # extract loss and accuracy from test data evaluation
print(score)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.8406 - auc: 0.7335 - val_loss: 0.7882 - val_auc: 0.8048
Epoch 2/10
1/1 [==============================] - 0s 46ms/step - loss: 0.8243 - auc: 0.7765 - val_loss: 0.7742 - val_auc: 0.8171
Epoch 3/10
1/1 [==============================] - 0s 33ms/step - loss: 0.8097 - auc: 0.8030 - val_loss: 0.7612 - val_auc: 0.8393
Epoch 4/10
1/1 [==============================] - 0s 34ms/step - loss: 0.7960 - auc: 0.8315 - val_loss: 0.7488 - val_auc: 0.8559
Epoch 5/10
1/1 [==============================] - 0s 31ms/step - loss: 0.7829 - auc: 0.8560 - val_loss: 0.7372 - val_auc: 0.8715
Epoch 6/10
1/1 [==============================] - 0s 32ms/step - loss: 0.7707 - auc: 0.8722 - val_loss: 0.7263 - val_auc: 0.8875
Epoch 7/10
1/1 [==============================] - 0s 32ms/step - loss: 0.7595 - auc: 0.8826 - val_loss: 0.7174 - val_auc: 0.8932
Epoch 8/10
1/1 [==============================] - 0s 46ms/step - loss: 0.7493 - auc: 0.8871 - val_l

In [21]:
#just so I know what my first model looks like
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrapp  (None, 3)                18        
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 3)                12        
 pper)                                                           
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


Try GridSearchCV to find the best number of hidden nodes.

In [52]:
#write this function
def create_model(hiddennodes=1):
	# create model
	model = Sequential()
	model.add(Dense(hiddennodes, activation="relu")) #layer 1
	model.add(Dense(3, activation="relu")) #layer 2; my second hidden layer must have 3 units since I have 3 categories
	# Compile model
	optimizer = SGD(learning_rate = 0.1)
	model.compile(loss='categorical_crossentropy',
	              optimizer=optimizer,
								       metrics='accuracy') #i ran it on accuracy instead of AUC because it didn't yield any results
	return model

model2 = KerasClassifier(build_fn = create_model, epochs = 25, verbose = 0)

#create my parameter grid
param_grid = dict(hiddennodes=[1,3,5,10,20])

#run GridSearch
grid = GridSearchCV(estimator=model2, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train)

In [53]:
#review the results of my gridsearch
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hiddennodes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.767175,0.010710,0.169256,0.006379,1,{'hiddennodes': 1},0.695652,0.173913,0.545455,0.590909,0.318182,0.464822,0.190690,5
1,0.756961,0.013510,0.166421,0.009112,3,{'hiddennodes': 3},0.608696,0.608696,1.000000,0.636364,0.681818,0.707115,0.148862,3
2,0.828539,0.184270,0.176117,0.011587,5,{'hiddennodes': 5},0.913043,0.478261,0.954545,0.681818,0.454545,0.696443,0.209697,4
3,0.731749,0.028377,0.177881,0.011645,10,{'hiddennodes': 10},0.695652,0.956522,0.590909,1.000000,0.363636,0.721344,0.236045,2
4,0.744904,0.027083,0.164974,0.006448,20,{'hiddennodes': 20},0.869565,0.565217,0.818182,1.000000,1.000000,0.850593,0.159684,1


Evaluate my predictions to determine the f1 score of my best model through GridSearchCV.

In [54]:
ypreds_classindex = grid_result.predict(X_test_scaled)

#transform my y_test into labels
y_test_converted = y_test.idxmax(axis=1)
y_test_converted = y_test_converted.replace("setosa", 0)
y_test_converted = y_test_converted.replace("versicolor", 1)
y_test_converted = y_test_converted.replace("virginica", 2)

In [55]:
print("Best performance on training data: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#evaluate predictions on average f1 score
f1_score_ypred = f1_score(y_test_converted, ypreds_classindex, average="macro")
print("Result on test set: {:.3f}".format(f1_score_ypred))

Best performance on training data: 0.850593 using {'hiddennodes': 20}
Result on test set: 0.924


In [57]:
%%shell
jupyter nbconvert --to html /content/HW_4_QMSS_5073.ipynb

[NbConvertApp] Converting notebook /content/HW_4_QMSS_5073.ipynb to html
[NbConvertApp] Writing 344188 bytes to /content/HW_4_QMSS_5073.html
